# Configuring Suites

In [1]:
# Following code is needed to preconfigure this notebook
import sys
import os
sys.path.insert(0, os.path.abspath('../../..'))

import pyflow as pf

scratchdir = os.path.join('/', 'path', 'to', 'scratch')
filesdir = os.path.join(scratchdir, 'files')
outdir = os.path.join(scratchdir, 'out')


class CourseSuite(pf.Suite):
    """
    This CourseSuite object will be used throughout the course to provide sensible
    defaults without verbosity
    """
    def __init__(self, name, **kwargs):
        
        config = {
            'host': pf.LocalHost(),
            'files': os.path.join(filesdir, name),
            'home': outdir,
            'defstatus': pf.state.suspended
        }
        config.update(kwargs)
        
        super().__init__(name, **config)


class MyTask(pf.Task):
    
    """Counts to the double of a number, first half using a for loop then a while loop"""
    
    def __init__(self, name, default_value=0, **kwargs):
        
        variables = {
            'HALF': default_value,
            'LIMIT': 2*default_value,
        }
        variables.update(**kwargs)
        
        labels = {
            'counter_label': 'count to {}'.format(2*default_value)
        }
        
        script = [
            'echo "This is a counting task named {}"'.format(name),
            'for i in $(seq 1 $HALF); do echo "count $i/$LIMIT"; done',
            'i=$[$HALF+1]; while [ $i -lt $LIMIT ]; do echo "count $i/$LIMIT" ; i=$[$i+1]; done'
        ]
        
        super().__init__(name,
                         script=script,
                         labels=labels,
                         **variables)

We build such library of classes and objects so we can re-use these components (Tasks, Families, Suites) in different contexts. A given task class could be used in a research workflow and then reused in another operational workflow.

However different contexts may require some differences in the suite execution. To ensure that we still have a concise, maintainable and easily checkable suite, we need to cater for those differences preferably in a single entity (as opposed to spreadout through the suite).

To that aim, we introduce the use of a _configuration object_ that will handle the differences, and therefore interact and configure our objects under each different context.

This results in suites that are _configurable_ for different use-cases and different contexts and build fundamentally different generated suites from the same components

A configuration object can be constructed manually for different use cases or as a result of parsing configuration files. It can be used to:

* Provide constants and data for specific cases, that will be needed in the suites.
* Switch functionality on/off or modify it.
* Configuration for hosts where to run the tasks.
* Locations of and details of data to process.
 
But most importantly, as objects, these configuration objects can be programmable in themselves (can include code). The suite components can delegate part of the suite definition to these _configurators_ and as such the structure of the suite can be determined by logic in the configuration object if necessary.

<div class="alert alert-warning">

Important

Delegation is preferred over conditional `if` statements in the suite depending on configuration values.

</div>

In [2]:
class BaseConfig:
    """This is a very contrived example showing delegation of behaviour to configuration"""
    
    def __init__(self, name, common_count=3, unit_count=4, integration_count=5):
        self.name = name
        self.common_count=common_count
        self.unit_count = unit_count
        self.integration_count = integration_count
    
    def build_unit_tests(self):
        pass
    
    def build_integration_tests(self):
        pass    
    
    
class ProductionConfig(BaseConfig): 
    def build_integration_tests(self):
        with pf.Family('integration') as f:
            pf.sequence(MyTask('integration_{}'.format(i), 123*i) for i in range(self.integration_count))
        return f
            
    
class DevConfig(BaseConfig):
    def build_unit_tests(self):
        with pf.Family('unit') as f:
            pf.sequence(MyTask('unit_{}'.format(i), 123*i) for i in range(self.unit_count))
        return f

We can now build a common testing family that behaves (structurally) differently according to the configuration supplied.

In [3]:
class ConfiguredFamily(pf.Family):
    def __init__(self, config):
        super().__init__(config.name)
        
        with self:

            # the static part of the suite, common to all suites of this type
            
            with pf.Family('common') as common:
                pf.sequence(MyTask('common_{}'.format(i), 123*i) for i in range(5))

            # the dynamic part of the suite, with hooks for the variability

            test_families = [
                config.build_unit_tests(),
                config.build_integration_tests()
            ]

            # some other static of the suite

            with pf.Family('cleanup') as cleanup:
                MyTask('cleaner')
            
            # establish dependencies
            
            common >> cleanup
            for f in test_families:
                if f is not None:
                    common >> f >> cleanup

In [4]:
with CourseSuite('configuration_example') as s:

    ConfiguredFamily(ProductionConfig('prod', integration_count=3))

    ConfiguredFamily(DevConfig('dev', unit_count=25))

s

CourseSuite(/configuration_example)